In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

22/03/09 16:29:26 WARN Utils: Your hostname, VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/09 16:29:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 16:29:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pyspark = spark.read.csv('test1.csv', header=True,inferSchema=True)
df_pyspark.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Agus|  31|        10| 30000|
| Budi|  30|         8| 25000|
|Cecep|  29|         4| 20000|
| Dede|  24|         3| 20000|
| Fani|  21|         1| 15000|
| Gaga|  23|         2| 18000|
| Heri|null|      null| 40000|
| null|  34|        10| 38000|
| null|  36|      null|  null|
+-----+----+----------+------+



In [4]:
# dropping any null valued row in data frame
df_pyspark_not_null = df_pyspark.na.drop(how="any")
df_pyspark_not_null.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Agus| 31|        10| 30000|
| Budi| 30|         8| 25000|
|Cecep| 29|         4| 20000|
| Dede| 24|         3| 20000|
| Fani| 21|         1| 15000|
| Gaga| 23|         2| 18000|
+-----+---+----------+------+



In [5]:
# dropping rows with minimal 2 non-null value in data frame
df_pyspark_threshold = df_pyspark.na.drop(how="any", thresh=2)
df_pyspark_threshold.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Agus|  31|        10| 30000|
| Budi|  30|         8| 25000|
|Cecep|  29|         4| 20000|
| Dede|  24|         3| 20000|
| Fani|  21|         1| 15000|
| Gaga|  23|         2| 18000|
| Heri|null|      null| 40000|
| null|  34|        10| 38000|
+-----+----+----------+------+



In [6]:
# dropping any null valued row in a subset column in data frame
df_pyspark_subset = df_pyspark.na.drop(how="any", subset=['Experience'])
df_pyspark_subset.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Agus| 31|        10| 30000|
| Budi| 30|         8| 25000|
|Cecep| 29|         4| 20000|
| Dede| 24|         3| 20000|
| Fani| 21|         1| 15000|
| Gaga| 23|         2| 18000|
| null| 34|        10| 38000|
+-----+---+----------+------+



In [7]:
# Filling null with something
df_pyspark_missing_values = df_pyspark.na.fill('Missing Values', subset=['Experience', 'Age'])
df_pyspark_missing_values.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Agus|  31|        10| 30000|
| Budi|  30|         8| 25000|
|Cecep|  29|         4| 20000|
| Dede|  24|         3| 20000|
| Fani|  21|         1| 15000|
| Gaga|  23|         2| 18000|
| Heri|null|      null| 40000|
| null|  34|        10| 38000|
| null|  36|      null|  null|
+-----+----+----------+------+



In [8]:
# Filling null with mean value

from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'],
    outputCols=["{}_imputed".format(c) for c in ['Age', 'Experience', 'Salary']]
    ).setStrategy("mean")

imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+----+----------+------+-----------+------------------+--------------+
| Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+------+-----------+------------------+--------------+
| Agus|  31|        10| 30000|         31|                10|         30000|
| Budi|  30|         8| 25000|         30|                 8|         25000|
|Cecep|  29|         4| 20000|         29|                 4|         20000|
| Dede|  24|         3| 20000|         24|                 3|         20000|
| Fani|  21|         1| 15000|         21|                 1|         15000|
| Gaga|  23|         2| 18000|         23|                 2|         18000|
| Heri|null|      null| 40000|         28|                 5|         40000|
| null|  34|        10| 38000|         34|                10|         38000|
| null|  36|      null|  null|         36|                 5|         25750|
+-----+----+----------+------+-----------+------------------+--------------+

In [9]:
spark.stop()